In [1]:
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout,Input,concatenate
from tensorflow.keras.callbacks import EarlyStopping
from datetime import timedelta, datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import time

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [89]:
recipes = pd.read_csv('keywordsfulldone.csv')

In [90]:
len(recipes)

52286

In [46]:
recipes = recipes.head(2000)
recipes.head()


,Unnamed: 0.1,Unnamed: 0,RecipeId,Name,AuthorId_x,AuthorName_x,CookTime,PrepTime,TotalTime,Description,...,FiberContent,SugarContent,ProteinContent,RecipeInstructions,ReviewId,AuthorId_y,AuthorName_y,Rating,Review,extracted_keywords
0,0,0,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,This is from one of my first Good House Keepi...,...,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ...",34854,59064,Missy Wombat,4.333333,My favourite lemonade recipe is essentially th...,"['My favourite lemonade recipe', 'this', 'the ..."
1,1,1,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,...,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil...",46368,71084,smurfy57,2.666667,This is a very good soup. We served ours warm...,"['This', 'a very good soup', 'We', 'ours', 'so..."
2,2,2,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,I copied this one out of a friend's book so ma...,...,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co...",14,2085,Tony Small,4.545455,An excellent dish.It is a great meal I put it ...,"['An excellent dish', 'It', 'a great meal', 'I..."
3,3,3,49,Chicken Breasts Lombardi,174711,Queen Dragon Mom,PT30M,PT45M,PT1H15M,Make and share this Chicken Breasts Lombardi r...,...,0.6,2.0,57.9,"c(""Cook mushrooms in 2 tbsp butter in a large ...",17145,19893,Luke Meakins,4.350000,I give this recipe a good to high rating becau...,"['I', 'this recipe', 'high rating', 'it', 'rea..."
4,4,4,54,Carrot Cake,1535,Marg CaymanDesigns,PT50M,PT45M,PT1H35M,This is one of the few recipes my husband ever...,...,1.6,47.9,5.0,"c(""Beat together the eggs, oil, and white suga...",74639,86318,mianbao,4.235294,"I made this according to the recipe, with no s...","['I', 'this', 'the recipe', 'no substitutions'..."


In [47]:
# import ast
# recipes['new'] = recipes['RecipeIngredientParts'].apply(ast.literal_eval)

# # Now convert the lists to strings
# recipes['new'] = recipes['new'].apply(lambda x: ', '.join(x))
# recipes["new"]

In [91]:
recipes.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'RecipeId', 'Name', 'AuthorId_x',
       'AuthorName_x', 'CookTime', 'PrepTime', 'TotalTime', 'Description',
       'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities',
       'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeInstructions', 'ReviewId', 'AuthorId_y',
       'AuthorName_y', 'Rating', 'Review', 'extracted_keywords'],
      dtype='object')

In [92]:
# Merge recipes and reviews dataframes
# merged_df = pd.merge(recipes, reviews, on='RecipeId')


In [93]:
from sklearn.preprocessing import LabelEncoder

# Encode RecipeId
# label_encoder = LabelEncoder()
# merged_df['EncodedRecipeId'] = label_encoder.fit_transform(merged_df['RecipeId'])
# Find the maximum label value in your dataset
# max_label_value = merged_df['RecipeId'].max()

# Increase the number of classes to accommodate the maximum label value
# num_classes = max_label_value + 1


In [94]:
#Language-agnostic BERT Sentence Embedding (LaBSE)
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('LaBSE')

In [95]:
recipes['Description'] = recipes['Description'].fillna("")
recipes['Name'] = recipes['Name'].fillna("")

recipes["text_features"] = recipes['Description'] + ". " + recipes['Name']+" . "+recipes['Review']
#recipes["text_features"]=recipes["extracted_keywords"]

# features = pd.DataFrame({
#     'text_features': recipes['text_features'],
#     'Calories': recipes['Calories']
# })
# features_array = features.values.astype(str)  # Convert to string type to avoid data type conflicts
# features_list = features_array.tolist()

In [96]:
embeddings = model.encode(recipes["text_features"], show_progress_bar=True, convert_to_tensor=True)


# for sentence, embedding in zip(recipes["text_features"], embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

Batches: 100%|██████████████████████████████| 1634/1634 [22:56<00:00,  1.19it/s]


In [114]:
# Function that searches the corpus and prints the results
def search(inp_question):
    start_time = time.time()
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, embeddings)
    end_time = time.time()
    hits = hits[0]  #Get the hits for the first query

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], recipes["text_features"][hit['corpus_id']]))
    return hits

In [115]:
hits=search("geberate cake recipe with chocolate and fruits")

Input question: geberate cake recipe with chocolate and fruits
Results (after 1.200 seconds):
	0.488	Make and share this Chocolate Cherry Cake recipe from Food.com.. Chocolate Cherry Cake . Really is a more of a cake then a bar.  But that aside this is very scrumptious cake - reminds me of a chocolate cover cherry with the wonderful chocolate frosting that is more like a thick glaze.  Highly recommended - my husband loved it!!This is a recipe I made for years and I am so happy I found it here again as I lost the recipe. I made this for a birthday at work and they went crazy for it. The cake batter is quite thick and you must mix carefully not to crush the fruit, but you will be rewarded with a moist cake and fudgy frosting that is killer with vanilla ice cream! A comforting dessert that brings rave reviews every time!I made this one this weekend and it was wonderful!! Because I wanted more of a bar than a cake (see PatD's review) I used a Betty Crocker Fudge Brownie mix. We were out of

In [116]:
recipes.iloc[hits[0]['corpus_id']]

Unnamed: 0.1                                                               3859
Unnamed: 0                                                                 3859
RecipeId                                                                  17795
Name                                                      Chocolate Cherry Cake
AuthorId_x                                                                26416
AuthorName_x                                                             duckit
CookTime                                                                  PT35M
PrepTime                                                                  PT15M
TotalTime                                                                 PT50M
Description                   Make and share this Chocolate Cherry Cake reci...
RecipeCategory                                                          Dessert
Keywords                                     c("< 60 Mins", "For Large Groups")
RecipeIngredientQuantities         c("1"

In [117]:
recipes.iloc[hits[0]['corpus_id']]['Name']

'Chocolate Cherry Cake'

In [111]:
import torch

# Assuming 'model' is your model
torch.save(model.state_dict(), './bertmodel/model.pth')
# Assuming 'embeddings' is your embeddings tensor
torch.save(embeddings, './bertmodel/embeddings.pth')


In [113]:
### howto load it for later
model.load_state_dict(torch.load('./bertmodel/model.pth'))
model.eval()  # Set the model to evaluation mode if it's for inference
embeddings = torch.load('./bertmodel/embeddings.pth')


In [17]:
# Define your search function
def search_abc(inp_question, max_calories):
    # Encode the input question
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    
    # Perform semantic search
    hits = util.semantic_search(question_embedding, embeddings)
    hits = hits[0]  # Get the hits for the first query

    # Filter hits based on calorie constraint
    filtered_hits = []
    for hit in hits:
        corpus_id = hit['corpus_id']
        if recipes.loc[corpus_id, 'Calories'] <= max_calories:
            filtered_hits.append(hit)

    # Display the results
    print("Input question:", inp_question)
    print("Results (with max {} calories):".format(max_calories))
    for hit in filtered_hits[0:10]:  # Display top 10 hits
        print("\t{:.3f}\t{}".format(hit['score'], recipes.loc[hit['corpus_id'], 'text_features']))
    return filtered_hits
hits=search_abc("Generate fish recipe ",100)

Input question: Generate fish recipe 
Results (with max 100 calories):
